In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [20]:
import bnl
import random

In [ ]:
tids = bnl.fio.salami_tids()
tid = random.choice(tids)
print(tid)

In [110]:
ests, refs = bnl.fio.salami_annos(tid)
est = next(iter(ests.values())).unique_labeling()
ref = next(iter(refs.values())).unique_labeling()
bnl.mtr.bmeasure(ref.itvls, est.itvls)

{'mu': 5,
 'ref_beta': 33,
 'est_beta': 16,
 'pairs_cap': 6,
 'ref_pairs': 8,
 'est_pairs': 6}

In [144]:
caps, ref_triplets, est_triplets = bnl.mtr.lmeasure(ref.itvls, ref.labels, est.itvls, est.labels)

In [145]:
print((caps / ref_triplets).sum(), (caps / est_triplets).sum())

0.5072909951632352 0.8571420481215314


In [133]:
ref_triplets

array([0.31597074, 0.31597074, 0.31597074, 0.31597074, 0.31597074,
       0.32527887, 0.33798985, 0.27307884, 0.27307884, 0.27383506,
       0.27383506, 0.27205618, 0.36425354, 0.35636537, 0.35636537,
       0.3452629 , 0.3452629 , 0.3452629 , 0.3455617 , 0.34531646,
       0.34531646, 0.34531646, 0.36121351, 0.36111059, 0.36111059,
       0.36085248, 0.27633314, 0.27633314, 0.27633314, 0.27633314,
       0.27726116, 0.27726116, 0.30882478, 0.30656454, 0.29551791,
       0.28860761, 0.28860761, 0.28860761, 0.28599917, 0.31663023,
       0.27884985, 0.27884985, 0.27884985, 0.27255453, 0.27130595,
       0.26715778])

In [128]:
caps.sum() / ref_triplets.sum(), caps.sum() / est_triplets.sum()


(0.010209586275926014, 0.02303002304567126)

In [129]:
(caps / ref_triplets).sum()

0.5072909951632352

In [130]:
(caps / est_triplets).sum()

0.8571420481215314

In [114]:
bnl.prf.compare_boundary_metrics('384')

Already computed 384.


<xarray.DataArray (tid: 1, anno_id: 2, component: 3, event: 2, window: 2)> Size: 192B
[24 values with dtype=float64]
Coordinates:
  * component  (component) object 24B 'cap' 'ref' 'est'
  * event      (event) object 16B 'beta' 'pair'
  * anno_id    (anno_id) int32 8B 0 1
  * window     (window) float64 16B 0.5 3.0
  * tid        (tid) object 8B '384'

({'mu': 9, 'beta': 39, 'pairs_cap': 9, 'pairs': 33},
 {'mu': 9, 'beta': 26, 'pairs_cap': 9, 'pairs': 20})